In [1]:
import datetime
import hashlib
import json
import requests
from flask import Flask, jsonify, request
from uuid import uuid4
from urllib.parse import urlparse

In [2]:
class Blockchain:
    
# Creating class constructor
    def __init__(self):                                     
        self.chain = []                                                 
        self.information = []                                           
        self.create_block(proof=1, previous_hash='0')
        self.nodes = set()

# Creating genesis block
    def create_block(self, proof, previous_hash):
        block = {
            'BLOCK NUMBER' : len(self.chain) + 1,
            'Timestamp'    : str(datetime.datetime.now()),
            'Proof'        : proof,
            'Previous Hash': previous_hash,
            'INFORMATION'  : self.information    
        } 
        self.information = []
# Adding genesis block to our chain
        self.chain.append(block)
        return block

# Function to return previous block
    def get_previous_block(self):  
        return self.chain[-1]

# Function for proof of work
    def proof_of_work(self, previous_proof):
        new_proof = 1
        check_proof = False
        while check_proof is False:
            hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
            if hash_operation[:4] == '0000':
                check_proof = True
            else:
                new_proof += 1
        return new_proof

# Hashing Block
    def hash(self, block):
        encoded_block = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(encoded_block).hexdigest()

# Checking Validity
    def is_chain_valid(self, chain):
        previous_block = chain[0]
        block_index = 1
        while block_index < len(chain):
            block = chain[block_index]
            if block['Previous Hash'] != self.hash(previous_block):
                return False
            previous_proof = previous_block['Proof']
            proof = block['Proof']
            hash_operation = hashlib.sha256(str(proof**2 - previous_proof**2).encode()).hexdigest()
            if hash_operation[:4] != '0000':
                return False
            previous_block = block
            block_index += 1
        return True


# Parsing Address
    def add_chain(self, address):
        parsed_url = urlparse(address)
        self.nodes.add(parsed_url.netloc)


    def replace_chain(self):
        network = self.nodes
        longest_chain = None
        max_length = len(self.chain)
        for node in network:
            response = requests.get(f'http://{node}/get_chain')
            if response.status_code == 200:
                length = response.json()['length']
                chain = response.json()['chain']
                if length > max_length and self.is_chain_valid(chain):
                    max_length = length
                    longest_chain = chain
            if longest_chain:
                self.chain = longest_chain
                return True
            return False

In [3]:
app = Flask(__name__)

In [4]:
node_address = str(uuid4()).replace('-', '')

In [5]:
blockchain = Blockchain()

In [6]:
@app.route('/mine_block', methods=['GET'])
def mine_block():
    previous_block = blockchain.get_previous_block()
    previous_proof = previous_block['Proof']
    proof = blockchain.proof_of_work(previous_proof)
    previous_hash = blockchain.hash(previous_block)
    block = blockchain.create_block(proof, previous_hash)
    response = {'message': 'MINING COMPLETE!',
                'BLOCK NUMBER': block['BLOCK NUMBER'],
                'Timestamp': block['Timestamp'],
                'Proof': block['Proof'],
                'Previous Hash': block['Previous Hash'],
                'INFORMATION': block['INFORMATION']}
    return jsonify(response), 200

In [7]:
@app.route('/get_chain', methods=['GET'])
def get_chain():
    response = {'chain': blockchain.chain,
                'length': len(blockchain.chain)
                }
    return jsonify(response), 200

In [8]:
@app.route('/is_valid', methods=['GET'])
def is_valid():
    is_valid = blockchain.is_chain_valid(blockchain.chain)
    if is_valid:
        response = {'message': 'VALID BLOCKCHAIN.'}
    else:
        response = {'BLOCKCHAIN NOT VALID'}
    return jsonify(response), 200

In [9]:
@app.route('/connect_chain', methods=['POST'])
def connect_chain():
    json = request.get_json()
    nodes = json.get('nodes')
    if nodes is None:
        return "No node", 400
    for node in nodes:
        blockchain.add_chain(node)
    response = {'message': 'All chains are now connected with other blockchain',
                'total_nodes': list(blockchain.nodes)}
    return jsonify(response), 201

In [10]:
@app.route('/replace_chain', methods=['GET'])
def replace_chain():
    is_chain_replaced = blockchain.replace_chain()
    if is_chain_replaced:
        response = {'message': 'Chain is replaced by larger one',
                    'new_chain': blockchain.chain}
    else:
        response = {'message': 'Everything fine. The chain is the longest one',
                    'actual_chain': blockchain.chain}
    return jsonify(response), 200

In [ ]:
app.run(host='0.0.0.0', port=5008)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5008/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Jan/2022 22:05:45] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2022 22:07:40] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2022 22:07:50] "POST /connect_chain HTTP/1.1" 201 -
127.0.0.1 - - [03/Jan/2022 22:47:54] "POST /replace_chain HTTP/1.1" 405 -
127.0.0.1 - - [03/Jan/2022 22:47:57] "GET /replace_chain HTTP/1.1" 200 -
